In [64]:
pip install networkx


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [65]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
import pandas as pd
import requests
import folium
import networkx as nx
import random
import math


df_preprocessing = pd.read_csv("airports-code@public.csv", sep=";")


df_preprocessing.head()

,Airport Code,Airport Name,City Name,Country Name,Country Code,Latitude,Longitude,World Area Code,City Name geo_name_id,Country Name geo_name_id,coordinates
0,CRS,Corsicana,Corsicana,United States,US,32.100000,-96.466667,67,4683462,6252001.0,"32.1, -96.466667"
1,ACP,Sahand,Sahand,Iran,IR,37.733333,46.450000,632,\N,130758.0,"37.733333, 46.45"
2,TBE,Timbunke,Timbunke,Papua New Guinea,PG,-4.183333,143.516667,804,\N,2088628.0,"-4.183333, 143.516667"
3,LWO,Snilow,Lviv,Ukraine,UA,49.812500,23.956111,488,702550,690791.0,"49.8125, 23.956111"
4,SCP,St Crepin,St Crepin,France,FR,44.683333,6.583333,427,\N,3017382.0,"44.683333, 6.583333"


In [51]:
# data preprossecing steps
df_preprocessing1 = df_preprocessing.drop(df_preprocessing.columns[[8,9]], axis = 1)
df_preprocessing1.head()

,Airport Code,Airport Name,City Name,Country Name,Country Code,Latitude,Longitude,World Area Code,coordinates
0,CRS,Corsicana,Corsicana,United States,US,32.100000,-96.466667,67,"32.1, -96.466667"
1,ACP,Sahand,Sahand,Iran,IR,37.733333,46.450000,632,"37.733333, 46.45"
2,TBE,Timbunke,Timbunke,Papua New Guinea,PG,-4.183333,143.516667,804,"-4.183333, 143.516667"
3,LWO,Snilow,Lviv,Ukraine,UA,49.812500,23.956111,488,"49.8125, 23.956111"
4,SCP,St Crepin,St Crepin,France,FR,44.683333,6.583333,427,"44.683333, 6.583333"


In [52]:
df_preprocessing[df_preprocessing.isnull().any(axis=1)]

,Airport Code,Airport Name,City Name,Country Name,Country Code,Latitude,Longitude,World Area Code,City Name geo_name_id,Country Name geo_name_id,coordinates
40,IDF,Idiofa,Idiofa,Zaire,ZR,-5.000000,19.508333,517,\N,NaN,"-5.0, 19.508333"
172,DGF,Bisbee-douglas International,Douglas,Unites States,US,31.468889,-109.653611,67,5823516,NaN,"31.468889, -109.653611"
289,LHR,London Heathrow,London,United Kingdom,UK,51.469722,-0.451389,493,\N,NaN,"51.469722, -0.451389"
302,INO,Inongo,Inongo,Democratic Republic of Congo,CD,-1.966667,18.266667,517,2315417,NaN,"-1.966667, 18.266666999999998"
307,USN,Ulsan,Ulsan,Reublic of Korea,KR,35.593611,129.352500,778,1833747,NaN,"35.593611, 129.3525"
...,...,...,...,...,...,...,...,...,...,...,...
8760,MSM,Masi Manimba,Masi Manimba,Zaire,ZR,-4.833333,17.850000,517,\N,NaN,"-4.833333, 17.85"
8906,HNO,Hercegnovi,Hercegnovi,Serbia and Montenegro,CS,42.566667,18.533333,497,\N,NaN,"42.566667, 18.533333"
8926,NKL,Nkolo,Nkolo,Zaire,ZR,-1.983333,16.466667,517,\N,NaN,"-1.983333, 16.466667"
9101,KGN,Kasongo Lunda,Kasongo Lunda,Zaire,ZR,-6.466667,16.816667,517,\N,NaN,"-6.466667, 16.816667"


In [53]:
def get_osrm_route(start_coords, end_coords):
    url = f"http://router.project-osrm.org/route/v1/driving/{start_coords[1]},{start_coords[0]};{end_coords[1]},{end_coords[0]}?overview=full&geometries=geojson"
    response = requests.get(url)
    
    if response.status_code == 200:
        route_data = response.json()
        distance_meters = route_data['routes'][0]['distance']  
        route_coords = route_data['routes'][0]['geometry']['coordinates']
        return route_coords, distance_meters
    else:
        return None, None


In [54]:
df_preprocessing1[['latitude', 'longitude']] = df_preprocessing1['coordinates'].str.split(',', expand=True)
df_preprocessing1['latitude'] = pd.to_numeric(df_preprocessing1['latitude'], errors='coerce')
df_preprocessing1['longitude'] = pd.to_numeric(df_preprocessing1['longitude'], errors='coerce')
df = df_preprocessing1.drop(df_preprocessing1.columns[[5,6]], axis = 1)
df.head()

,Airport Code,Airport Name,City Name,Country Name,Country Code,World Area Code,coordinates,latitude,longitude
0,CRS,Corsicana,Corsicana,United States,US,67,"32.1, -96.466667",32.100000,-96.466667
1,ACP,Sahand,Sahand,Iran,IR,632,"37.733333, 46.45",37.733333,46.450000
2,TBE,Timbunke,Timbunke,Papua New Guinea,PG,804,"-4.183333, 143.516667",-4.183333,143.516667
3,LWO,Snilow,Lviv,Ukraine,UA,488,"49.8125, 23.956111",49.812500,23.956111
4,SCP,St Crepin,St Crepin,France,FR,427,"44.683333, 6.583333",44.683333,6.583333


In [ ]:

chosen_airport= ['Tokyo Haneda International','Soekarno-Hatta International', 'John F Kennedy Intl', 'Sydney']
df_chosen_airports = df[df['Airport Name'].isin(chosen_airport)]
print(df_chosen_airports)
m = folium.Map(location=[df_chosen_airports['latitude'].mean(), df_chosen_airports['longitude'].mean()], zoom_start=3)

airport_locations = list(zip(df_chosen_airports['latitude'], df_chosen_airports['longitude']))
airport_names = df_chosen_airports['Airport Name'].tolist()

for index, row in df_chosen_airports.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['Airport Name']}<br>{row['City Name']}<br>{row['Country Name']}",).add_to(m)



     Airport Code                  Airport Name   City Name   Country Name  \
2267          YQY                        Sydney  Sydney, NS         Canada   
3443          CGK  Soekarno-Hatta International     Jakarta      Indonesia   
4000          JFK           John F Kennedy Intl    New York  United States   
4662          HND    Tokyo Haneda International       Tokyo          Japan   

     Country Code  World Area Code            coordinates   latitude  \
2267           CA              906  47.528056, -60.047222  47.528056   
3443           ID              832  -6.125556, 106.655833  -6.125556   
4000           US               67  40.633333, -73.783333  40.633333   
4662           JP              736  35.552222, 139.779722  35.552222   

       longitude  
2267  -60.047222  
3443  106.655833  
4000  -73.783333  
4662  139.779722  


In [ ]:
G = nx.Graph()

size = df_chosen_airports.shape[0]
for i in range(size):
    G.add_node(airport_names[i], pos=airport_locations[i])

from geopy.distance import geodesic

total_distance = 0
for i in range(len(airport_locations) - 1):
    for j in range(i + 1, len(airport_locations)): 
        start_coords = airport_locations[i]
        end_coords = airport_locations[j]
        
        # Compute geodesic distance in kilometers
        distance_km = geodesic(start_coords, end_coords).km
        total_distance += distance_km

        # Draw a straight line on the map
        route = [(start_coords[0], start_coords[1]), (end_coords[0], end_coords[1])]
        folium.PolyLine(locations=[route], color='blue', weight=2.5, opacity=0.8).add_to(m)
        
        # Add edge to the graph with distance as weight
        G.add_edge(airport_names[i], airport_names[j], weight=distance_km)

# Display the map
m


In [81]:

from queue import PriorityQueue 
import heapq

In [106]:


def addEdge(node, visited, pq, graph):
    visited[node] = True

    edges = graph.edges(node)
    for neighbor, edge_data in graph[node].items():
        if not visited[neighbor]:
            pq.put((edge_data['weight'], node, neighbor, graph.nodes[node]['pos'], graph.nodes[neighbor]['pos']))
        
    

def prims(graph, start):
    n = len(graph.nodes)
    visited = {node: False for node in graph.nodes} 
    m = graph.number_of_edges()
    edgeCount, mstCost = 0, 0
    mstEdges = []
    pq = PriorityQueue()

    addEdge(start, visited, pq, graph)


    while not pq.empty() and edgeCount < m:
        edge, startNode, endNode, startCoords, endCoords = pq.get()

        if visited[endNode] == True:
            continue
        
        visited[endNode] = True
        mstEdges.append((startNode, endNode, edge, startCoords, endCoords))
        edgeCount = edgeCount + 1
        mstCost = mstCost + edge

        addEdge(endNode, visited, pq, graph)

    return mstEdges, mstCost


In [ ]:
mst_edges, mst_cost = prims(G, airport_names[0]) 

print(mst_edges)


m_mst = folium.Map(location=[df_chosen_airports['latitude'].mean(), df_chosen_airports['longitude'].mean()], zoom_start=3)

for index, row in df_chosen_airports.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"{row['Airport Name']}<br>{row['City Name']}<br>{row['Country Name']}",).add_to(m_mst)


total_distance_mst = 0

count = 0
for edge in mst_edges:
    startNode, endNode, weight, startCoords, endCoords = edge

    route = [startCoords, endCoords]

    folium.PolyLine(locations=route, color='blue', weight=2.5, opacity=0.8).add_to(m_mst)
    
    total_distance_mst += weight
    count += 1

print(f"Total Distance of MST: {total_distance_mst} km")
print(count)

m_mst

[('Sydney', 'John F Kennedy Intl', 1337.5399067983637, (47.528056, -60.047222), (40.633333, -73.783333)), ('Sydney', 'Tokyo Haneda International', 10593.205917816274, (47.528056, -60.047222), (35.552222, 139.779722)), ('Tokyo Haneda International', 'Soekarno-Hatta International', 5770.022311061451, (35.552222, 139.779722), (-6.125556, 106.655833))]
Total Distance of MST: 17700.768135676088 km
3
